In [0]:
filepath = ""
output_filepath = ""

In [0]:
#!pip install pydub
#!pip install google-cloud-speech
from google.cloud import speech_v1p1beta1 as speech

In [0]:
from pydub import AudioSegment
import io
import os
#from google.cloud import speech
from google.cloud import speech_v1p1beta1 as speech

#from google.cloud.speech import enums
#from google.cloud.speech import types

from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types

import wave
from google.cloud import storage
import requests
import shutil
import pandas as pd

In [0]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'google_text_to_speech_key.json'

In [0]:
df = pd.read_csv('all_podcasts.csv')
df['mp3_filename'] =df['data-asset-source'].str.split('\/').str[-1]
df = df[~df.mp3_filename.isnull()]
#df['data-asset-source'].tolist()

In [0]:
df[df['data-asset-title'].str.contains('Haneke')]

,Unnamed: 0,index,Unnamed: 0.1,aria-label,class,data-asset-created-date,data-asset-source,data-asset-surtitle,data-asset-title,data-btn-type,data-business-reference,data-clip-type,data-concept-id,data-diffusion-id,data-duration,data-expression-id,data-media-id,data-network-change,data-short-id,data-surtitle-link,data-timeshift,data-title-link,data-universe-title,disabled,title,mp3_filename
1480,1480,1480,10,Lecture du replay,replay-button paused aod,1.358413e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Philosopher avec Michael Haneke (4/4) : La pia...,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3567.0,08f3b1af-5b36-11e2-a7b7-782bcb76618d,08f3b1af-5b36-11e2-a7b7-782bcb76618d,NaN,0a152555,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Philosopher avec Michael Haneke (4/4...,RF_6A1DC2F7-A165-421B-B7FF-63067F201514_GENE.MP3
1481,1481,1481,11,Lecture du replay,replay-button paused aod,1.358327e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Philosopher avec Michael Haneke (3/4) : Funny ...,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3495.0,327ba26c-5b35-11e2-a7b7-782bcb76618d,327ba26c-5b35-11e2-a7b7-782bcb76618d,NaN,fbe4fe4c,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Philosopher avec Michael Haneke (3/4...,NET_FC_30b633f9-dc13-4b97-bcbd-ef3defc01bad.mp3
1482,1482,1482,12,Lecture du replay,replay-button paused aod,1.358240e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Philosopher avec Michael Haneke (2/4) : Le rub...,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3570.0,d62a0b09-5b34-11e2-a7b7-782bcb76618d,d62a0b09-5b34-11e2-a7b7-782bcb76618d,NaN,2020d74f,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Philosopher avec Michael Haneke (2/4...,PHILOSOPHER_AVEC_MICHAEL_HANEKE_%282_4%29___LE...
1483,1483,1483,13,Lecture du replay,replay-button paused aod,1.358154e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Philosopher avec Michael Haneke (1/4) : Le cin...,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3483.0,7bb10c3c-5b34-11e2-a7b7-782bcb76618d,7bb10c3c-5b34-11e2-a7b7-782bcb76618d,NaN,ce12d2aa,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Philosopher avec Michael Haneke (1/4...,NET_FC_f21b5923-d17a-4444-8ee3-324de55377ed.mp3


In [0]:
def dl_mp3(file_url,file_name):
  r = requests.get(file_url, stream=True)
  if r.status_code == 200:
      with open(file_name, 'wb') as f:
          r.raw.decode_content = True
          shutil.copyfileobj(r.raw, f)        

In [0]:
def mp3_to_wav(mp3_file_name):
    print('converting')
    wav_file_name = mp3_file_name.split('.mp3')[0] + '.wav'
    exists = os.path.isfile(output_filepath + mp3_file_name)
    if not exists:
      
      sound = AudioSegment.from_mp3(mp3_file_name)
      print('sound ok')
    
      print(wav_file_name)
      sound.export(wav_file_name, format="wav")
    return wav_file_name

In [0]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [0]:
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [0]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [0]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [0]:
def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
    print(file_name)
    wav_file_name = mp3_to_wav(file_name)
    print(wav_file_name)

    # The name of the audio file to transcribe
    
    frame_rate, channels = frame_rate_channel(wav_file_name)
    
    if channels > 1:
        stereo_to_mono(wav_file_name)
    
    bucket_name = 'callsaudiofiles_clem'
    source_file_name = filepath + wav_file_name
    destination_blob_name = wav_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://callsaudiofiles_clem/' + wav_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='fr-FR',
    enable_speaker_diarization=True,
    diarization_speaker_count=2)

    # Detects speech in the audio file
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)
    result = response.results[-1]
    words_info = result.alternatives[0].words
    
    tag=1
    speaker=""

    for word_info in words_info:
        if word_info.speaker_tag==tag:
            speaker=speaker+" "+word_info.word
        else:
            transcript += "speaker {}: {}".format(tag,speaker) + '\n'
            tag=word_info.speaker_tag
            speaker=""+word_info.word

    transcript += "speaker {}: {}".format(tag,speaker)
    
    #delete_blob(bucket_name, destination_blob_name)
    return transcript

In [0]:
def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+")
    f.write(transcript)
    f.close() 

In [0]:
if __name__ == "__main__":
  for index, row in df.head(-1).iterrows():
    print(row['mp3_filename'], row['data-asset-source'])
    #dl_mp3(row['data-asset-source'],row['mp3_filename'])
     
    transcript = google_transcribe(row['mp3_filename'])
    transcript_filename = row['mp3_filename'].split('.mp3')[0] + '.txt'
    write_transcripts(transcript_filename,transcript)

18755-12.07.2019-ITEMA_22108495-0.mp3 https://media.radiofrance-podcast.net/podcast09/18755-12.07.2019-ITEMA_22108495-0.mp3
18755-12.07.2019-ITEMA_22108495-0.mp3
converting
18755-12.07.2019-ITEMA_22108495-0.wav
18755-11.07.2019-ITEMA_22107864-0.mp3 https://media.radiofrance-podcast.net/podcast09/18755-11.07.2019-ITEMA_22107864-0.mp3
18755-11.07.2019-ITEMA_22107864-0.mp3
converting


FileNotFoundError: ignored